In [1]:
!python -m pip uninstall matplotlib

Found existing installation: matplotlib 3.6.3
Uninstalling matplotlib-3.6.3:
  Would remove:
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/matplotlib-3.6.3-py3.9-nspkg.pth
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/matplotlib-3.6.3.dist-info/*
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/matplotlib/*
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/mpl_toolkits/axes_grid1/*
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/mpl_toolkits/axisartist/*
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/mpl_toolkits/mplot3d/*
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/mpl_toolkits/tests/*
    /Users/laurenchan/.virtualenvs/expo_env/lib/python3.9/site-packages/pylab.py
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [4]:

!pip install matplotlib==3.1.3 -U
!pip install -qU grape

  Using cached matplotlib-3.1.3.tar.gz (40.9 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [503 lines of output]
      Edit setup.cfg to change the build options
      
      BUILDING MATPLOTLIB
        matplotlib: yes [3.1.3]
            python: yes [3.9.10 (main, Jan 15 2022, 11:48:00)  [Clang 13.0.0
                        (clang-1300.0.29.3)]]
          platform: yes [darwin]
      
      OPTIONAL SUBPACKAGES
       sample_data: yes [installing]
             tests: no  [skipping due to configuration]
      
      OPTIONAL BACKEND EXTENSIONS
               agg: yes [installing]
             tkagg: yes [installing; run-time loading from Python Tcl/Tk]
            macosx: yes [installing, darwin]
      
      OPTIONAL PACKAGE DATA
              dlls: no  [skipping due to configuration]
      
      running bdist_wheel
      running build
      running build_py
     

      copying lib/matplotlib/mpl-data/stylelib/fast.mplstyle -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/stylelib
      copying lib/matplotlib/mpl-data/fonts/afm/pncr8a.afm -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/fonts/afm
      copying lib/matplotlib/mpl-data/fonts/afm/phvb8a.afm -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/fonts/afm
      copying lib/matplotlib/mpl-data/stylelib/seaborn-dark-palette.mplstyle -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/stylelib
      copying lib/matplotlib/mpl-data/stylelib/fivethirtyeight.mplstyle -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/stylelib
      copying lib/matplotlib/mpl-data/fonts/ttf/STIXNonUniBol.ttf -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/fonts/ttf
      creating build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/sample_data/axes_grid
      copying lib/matplotlib/mpl-data/sample_data/axes_grid/bivariate_normal.npy -> build/lib.macosx-11-x86_64-3.9/matplotlib/mpl-data/sa

Failed to build matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.6.3
    Uninstalling matplotlib-3.6.3:
      Successfully uninstalled matplotlib-3.6.3
  Running setup.py install for matplotlib ... error
  error: subprocess-exited-with-error
  
  × Running setup.py install for matplotlib did not run successfully.
  │ exit code: 1
  ╰─> [503 lines of output]
      Edit setup.cfg to change the build options
      
      BUILDING MATPLOTLIB
        matplotlib: yes [3.1.3]
            python: yes [3.9.10 (main, Jan 15 2022, 11:48:00)  [Clang 13.0.0
                        (clang-1300.0.29.3)]]
          platform: yes [darwin]
      
      OPTIONAL SUBPACKAGES
       sample_data: yes [installing]
             tests: no  [skipping due to configuration]
      
      OPTIONAL BACKEND EXTENSIONS
               agg: yes [installing]
             tkagg: yes [installing; run-time loading from Python Tcl/Tk]
            macosx: yes [installing, darwin]
   

In [ ]:
!tar -xvzf merged-kg.tar.gz
merged_node_filename='merged-kg_nodes.tsv'
merged_edge_filename='merged-kg_edges.tsv'

In [ ]:
# this is a very high degree node that isn't meaningful for node embeddings
# Definition: A term that is metadata complete, has been reviewed, and problems have been identified that require discussion before release. Such a term requires editor note(s) to identify the outstanding issue
!grep -v IAO:0000428 merged-kg_edges.tsv > tmp
!mv tmp merged-kg_edges.tsv

In [ ]:
from grape import Graph

In [ ]:
g = Graph.from_csv(
    node_path=merged_node_filename,
    edge_path=merged_edge_filename,
    node_list_separator="\t",
    edge_list_separator="\t",
    node_list_header=True,  # Always true for KG-Hub KGs
    edge_list_header=True,  # Always true for KG-Hub KGs
    nodes_column='id',  # Always true for KG-Hub KGs
    node_list_node_types_column='category',  # Always true for KG-Hub KGs
    sources_column='subject',  # Always true for KG-Hub KGs
    destinations_column='object',  # Always true for KG-Hub KGs
    directed=False,
    name="kg-exposure",
    verbose=True
)
g

In [ ]:
# select largest components
g = g.remove_disconnected_nodes()
g = g.remove_components(top_k_components=1)
#connected component , pick the biggest connected component, the space where there is a large connection, needs the main connected component, won't be 
#able to look at stuff that is beyond the main component since it doesn't have that much info 

In [ ]:
from grape.embedders import FirstOrderLINEEnsmallen
embedding = FirstOrderLINEEnsmallen().fit_transform(g)

In [ ]:
from grape import GraphVisualizer
visualizer = GraphVisualizer(g)

In [ ]:
# You can either provide the model name
#visualizer.fit_nodes("First-order LINE", library_name="Ensmallen")
# Or provide a precomputed embedding
#
# visualizer.fit_nodes(numpy_array_with_embedding)
# visualizer.fit_nodes(pandas_dataframe_with_embedding)
#
# or alternatively provide the model to be used:
#
# from grape.embedders import FirstOrderLINEEnsmallen
# visualizer.fit_nodes(FirstOrderLINEEnsmallen())
#
# In this tutorial, we use the embedding we have just computed above:

visualizer.fit_nodes(embedding)

# And now we can visualize the node types:
visualizer.plot_node_types()



In [ ]:
visualizer.plot_node_degree_distribution()

# lots of nodes with very few neighbors

In [ ]:
visualizer.fit_and_plot_all(embedding)

In [ ]:
from grape.edge_prediction import RandomForestEdgePrediction
#from grape.embedders import FirstOrderLINEEnsmallen
#from grape.datasets.string import HomoSapiens

In [ ]:
# %%time
# graph = HomoSapiens()\
#     .filter_from_ids(min_edge_weight=700)\
#     .remove_disconnected_nodes()

In [ ]:
train, test = g.connected_holdout(train_size=0.7)

In [ ]:
%%time
embedding = FirstOrderLINEEnsmallen().fit_transform(train)

In [ ]:
%%time
model = RandomForestEdgePrediction()
model.fit(
    graph=train, 
    node_features=embedding
)

In [ ]:
[
    method_name
    for method_name in dir(model)
    if method_name.startswith("predict")
]

In [ ]:
%%time
# A perfect model should correctly predict the existance
# of all of these edges.
model.predict_proba(
    graph=test,
    node_features=embedding,
    return_predictions_dataframe=True
)

In [ ]:
%%time
# A perfect model should correctly predict the non-existance
# of all of these edges.
model.predict_proba(
    graph=graph.sample_negative_graph(number_of_negative_samples=test.get_number_of_edges()),
    node_features=embedding,
    return_predictions_dataframe=True
)